In [1]:
import pandas as pd
import os 
import glob

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import re
import requests
import json
import yfinance as yf
from time import sleep
from tqdm import tqdm
from datetime import datetime as dt

In [3]:
pd.set_option('display.max_rows', 100)

Read in Scraped Earnings Call Data

In [5]:
# pattern to match all CSVs
files = glob.glob("earnings_calls/*/scraped_earnings_calls.csv")

dfs = []
for f in files:
    ticker = os.path.basename(os.path.dirname(f))  # folder name is the ticker
    df = pd.read_csv(f, index_col = 0)
    df["ticker"] = ticker
    dfs.append(df)

# combine into one DataFrame
all_calls = pd.concat(dfs, ignore_index = True)
all_calls = all_calls.sort_values('ticker').reset_index(drop = True)



In [7]:
problematic_data = all_calls[
    all_calls["earnings_call_raw_text"].str.contains("Toggle navigation menu\nSearch", na=False)
]
problematic_data

,year_quarter,earnings_call_raw_text,ticker,date
23,2025-year/4-quarter,Toggle navigation menu\nSearch\n⌘\nK\nToggle t...,A,2025-10-01
24,2025-year/3-quarter,Toggle navigation menu\nSearch\n⌘\nK\nToggle t...,A,2025-07-01
66,2025-year/3-quarter,Toggle navigation menu\nSearch\n⌘\nK\nEarn\nTo...,AAON,2025-07-01
67,2025-year/4-quarter,Toggle navigation menu\nSearch\n⌘\nK\nEarn\nTo...,AAON,2025-10-01
982,2024-year/1-quarter,Toggle navigation menu\nSearch\n⌘\nK\nEarn\nTo...,ALE,2024-01-01
...,...,...,...,...
23497,2025-year/3-quarter,Toggle navigation menu\nSearch\n⌘\nK\nToggle t...,ZBH,2025-07-01
23540,2025-year/3-quarter,Toggle navigation menu\nSearch\n⌘\nK\nToggle t...,ZBRA,2025-07-01
23583,2025-year/4-quarter,Toggle navigation menu\nSearch\n⌘\nK\nToggle t...,ZBRA,2025-10-01
23603,2025-year/3-quarter,Toggle navigation menu\nSearch\n⌘\nK\nToggle t...,ZTS,2025-07-01


In [46]:
url = f"https://www.roic.ai/quote/DE/transcripts/2025-year/3-quarter"
text = get_earnings_call_text(url)

In [47]:
text

"\nAug 14, 2025\nOperator\nGood morning, and welcome to Deere & Company's Third Quarter Earnings Conference Call. [Operator Instructions] I would now like to turn the call over to Mr.\nJosh Beal, Director of Investor Relations. Thank you.\nYou may begin.\nJosh Beal\nHello. Welcome, and thank you for joining us on today's call.\nJoining me on the call today are Josh Jepsen, Chief Financial Officer; Cory Reed, President, Worldwide Agriculture and Turf Division, Production and Precision Ag, Americas and Australia; and Chris Seibert, Manager, Investor Communications. Today, we'll take a closer look at Deere's third quarter earnings, then spend some time talking about our end markets and our current outlook for fiscal 2025.\nAfter that, we'll respond to your questions. Please note that slides are available to complement the call this morning.\nThey can be accessed on our website at johndeere.com/ earnings. First, a reminder, this call is broadcast live on the Internet and recorded for futur

In [53]:
all_calls.loc[5677].earnings_call_raw_text = text

In [54]:
all_calls.loc[5677]

year_quarter                                            2025-year/3-quarter
earnings_call_raw_text    \nAug 14, 2025\nOperator\nGood morning, and we...
ticker                                                                   DE
date                                                             2025-07-01
Name: 5677, dtype: object

In [58]:
all_calls

,year_quarter,earnings_call_raw_text,ticker,date
0,2015-year/2-quarter,"\nMay 19, 2015\nExecutives\nAlicia Rodriguez -...",A,2015-04-01
1,2019-year/4-quarter,"\nNov 26, 2019\nOperator\nGood afternoon, and ...",A,2019-10-01
2,2019-year/3-quarter,"\nAug 14, 2019\nOperator\nGood afternoon, and ...",A,2019-07-01
3,2019-year/2-quarter,"\nMay 14, 2019\nOperator\nGood day, ladies and...",A,2019-04-01
4,2019-year/1-quarter,"\nFeb 20, 2019\nOperator\nGood day, ladies and...",A,2019-01-01
...,...,...,...,...
22127,2019-year/2-quarter,"\nAug 6, 2019\nOperator\nWelcome to the Second...",ZTS,2019-04-01
22128,2019-year/3-quarter,"\nNov 7, 2019\nOperator\nWelcome to the Third ...",ZTS,2019-07-01
22129,2019-year/4-quarter,"\nFeb 13, 2020\nOperator\nGood day. And welcom...",ZTS,2019-10-01
22130,2020-year/2-quarter,"\nAug 6, 2020\nOperator\nWelcome to the Second...",ZTS,2020-04-01


In [8]:
all_calls = all_calls.dropna()

In [9]:
all_calls.to_csv('all_calls.csv')

In [43]:
# problematic_data.index
for ind in problematic_data.index:
    year_quarter = problematic_data.loc[ind].year_quarter
    ticker = problematic_data.loc[ind].ticker

    url = f"https://www.roic.ai/quote/{ticker}/transcripts/{year_quarter}"

    if ticker[0] in ['A', 'B', 'C']:
        continue

    if ticker == 'DASH':
        continue
    try:
        text = get_earnings_call_text(url)
    except Exception as e:
        print(e)
        text = ''
    if not 'Transcript not found' in text:
        print(url)


# all_calls.loc[22021]['earnings_call_raw_text'] = 'clean'



https://www.roic.ai/quote/DE/transcripts/2025-year/3-quarter
https://www.roic.ai/quote/NTRS/transcripts/2018-year/1-quarter
https://www.roic.ai/quote/NTRS/transcripts/2018-year/2-quarter
https://www.roic.ai/quote/NTRS/transcripts/2018-year/3-quarter
https://www.roic.ai/quote/NTRS/transcripts/2019-year/1-quarter
https://www.roic.ai/quote/NTRS/transcripts/2019-year/3-quarter
https://www.roic.ai/quote/NTRS/transcripts/2019-year/2-quarter
https://www.roic.ai/quote/NTRS/transcripts/2017-year/4-quarter
https://www.roic.ai/quote/NTRS/transcripts/2018-year/4-quarter
https://www.roic.ai/quote/NTRS/transcripts/2017-year/3-quarter
https://www.roic.ai/quote/NTRS/transcripts/2015-year/2-quarter
https://www.roic.ai/quote/NTRS/transcripts/2017-year/1-quarter
https://www.roic.ai/quote/NTRS/transcripts/2016-year/4-quarter
https://www.roic.ai/quote/NTRS/transcripts/2016-year/3-quarter
https://www.roic.ai/quote/NTRS/transcripts/2016-year/2-quarter
https://www.roic.ai/quote/NTRS/transcripts/2016-year/1-qu